In [1]:
#load training data

from data_preprocess.adult import load_adult

train_data, test_data, data_info,cat_vars = load_adult()

Data info: [(1, 'tanh', 'Age'), (7, 'softmax', 'WorkClass'), (16, 'softmax', 'EducationNum'), (2, 'softmax', 'MaritalStatus'), (14, 'softmax', 'Occupation'), (6, 'softmax', 'Relationship'), (5, 'softmax', 'Race'), (2, 'softmax', 'Gender'), (3, 'softmax', 'CapitalGain'), (3, 'softmax', 'CapitalLoss'), (45, 'softmax', 'HoursPerWeek'), (2, 'softmax', 'NativeCountry'), (2, 'softmax', 'Income')]


In [2]:
train_data.head()

,Age,WorkClass,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
33716,27,Local-gov,10,Couple,Adm-clerical,Own-child,White,Female,zero,zero,17.0,US,<=50K
16224,50,Private,9,Single,Craft-repair,Unmarried,White,Female,zero,zero,17.0,US,<=50K
24357,50,Private,9,Single,Craft-repair,Unmarried,White,Male,zero,low,22.0,US,<=50K
44698,42,Private,9,Single,Exec-managerial,Not-in-family,White,Female,zero,zero,17.0,US,<=50K
37191,20,Private,6,Single,Craft-repair,Own-child,White,Male,zero,zero,17.0,US,<=50K


In [3]:
#train GAN
# support the follwoing net:
#    'WGANGP',
#    'dp_WGanSynthesizer', differentially private wgan with weight clipping
#    'CTGANSynthesizer',
#    'TVAESynthesizer',

from Synthesizers import *
from Synthesizers.train import train_Gmodel,sample_data_fromGen
gan,tf = train_Gmodel(WGANGP,train_data,data_info, seed=0,epochs=300,bs=500)

data dim: 108
Begin training...
[0/300]	  Loss_D: -0.2193	Loss_G: -0.7213	
[1/300]	  Loss_D: -0.1343	Loss_G: -0.4465	
[2/300]	  Loss_D: -0.0068	Loss_G: -0.1306	
[3/300]	  Loss_D: -0.0121	Loss_G: -0.2854	
[4/300]	  Loss_D: -0.0609	Loss_G: -0.2253	
[5/300]	  Loss_D: -0.0669	Loss_G: -0.2320	
[6/300]	  Loss_D: -0.0746	Loss_G: -0.3054	
[7/300]	  Loss_D: -0.0419	Loss_G: -0.3249	
[8/300]	  Loss_D: -0.0471	Loss_G: -0.3157	
[9/300]	  Loss_D: -0.0423	Loss_G: -0.3361	
[10/300]	  Loss_D: -0.0739	Loss_G: -0.2752	
[11/300]	  Loss_D: -0.0529	Loss_G: -0.2931	
[12/300]	  Loss_D: -0.0783	Loss_G: -0.2701	
[13/300]	  Loss_D: -0.0493	Loss_G: -0.2739	
[14/300]	  Loss_D: -0.0416	Loss_G: -0.2149	
[15/300]	  Loss_D: -0.0748	Loss_G: -0.2313	
[16/300]	  Loss_D: -0.0836	Loss_G: -0.1874	
[17/300]	  Loss_D: -0.0690	Loss_G: -0.2488	
[18/300]	  Loss_D: -0.0230	Loss_G: -0.2577	
[19/300]	  Loss_D: -0.0466	Loss_G: -0.2864	
[20/300]	  Loss_D: -0.0512	Loss_G: -0.2665	
[21/300]	  Loss_D: -0.0483	Loss_G: -0.3317	
[22/300]	 

In [4]:
syn = sample_data_fromGen(gan.myG.cpu(),tf,n=int(len(train_data)),data_info=data_info,seed=0)

Begin sample，seed= 0
Inverse transform completed!


In [5]:
syn.head()

,Age,WorkClass,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,20.0,Private,9.0,Single,Handlers-cleaners,Own-child,White,Male,zero,zero,13.0,US,<=50K
1,32.0,Private,7.0,Couple,Transport-moving,Husband,White,Male,zero,zero,17.0,US,<=50K
2,40.0,Private,6.0,Couple,Craft-repair,Husband,White,Male,zero,zero,17.0,US,<=50K
3,48.0,Private,10.0,Couple,Handlers-cleaners,Husband,White,Male,zero,zero,32.0,US,<=50K
4,53.0,Private,10.0,Single,Craft-repair,Unmarried,White,Female,zero,zero,17.0,US,>50K


In [6]:
from attacks.TableGAN_MCA import Tablegan_mca
labeled_syn = Tablegan_mca(train_data,
                            syn,
                            data_info,
                            WGANGP,
                            epochs=300, 
                            seed=0)

Synthetic datasets info: [(1, 'tanh', 'Age'), (7, 'softmax', 'WorkClass'), (16, 'softmax', 'EducationNum'), (2, 'softmax', 'MaritalStatus'), (13, 'softmax', 'Occupation'), (6, 'softmax', 'Relationship'), (5, 'softmax', 'Race'), (2, 'softmax', 'Gender'), (3, 'softmax', 'CapitalGain'), (3, 'softmax', 'CapitalLoss'), (41, 'softmax', 'HoursPerWeek'), (2, 'softmax', 'NativeCountry'), (2, 'softmax', 'Income')]
data dim: 103
Begin training...
[0/300]	  Loss_D: -0.0275	Loss_G: -0.6218	
[1/300]	  Loss_D: -0.4065	Loss_G: -0.2754	
[2/300]	  Loss_D: -0.0124	Loss_G: -0.2558	
[3/300]	  Loss_D: -0.0234	Loss_G: -0.2366	
[4/300]	  Loss_D: -0.0563	Loss_G: -0.1474	
[5/300]	  Loss_D: -0.1007	Loss_G: -0.1724	
[6/300]	  Loss_D: -0.0489	Loss_G: -0.1061	
[7/300]	  Loss_D: -0.0270	Loss_G: -0.2146	
[8/300]	  Loss_D: -0.0441	Loss_G: -0.2505	
[9/300]	  Loss_D: -0.0647	Loss_G: -0.3965	
[10/300]	  Loss_D: -0.0245	Loss_G: -0.3396	
[11/300]	  Loss_D: -0.0367	Loss_G: -0.2826	
[12/300]	  Loss_D: -0.0568	Loss_G: -0.4007

In [7]:
y_test =labeled_syn['label'].values
y_score =labeled_syn['y_score'].values

In [8]:
from sklearn.metrics import precision_recall_curve, roc_auc_score
AUCROC = roc_auc_score(y_test, y_score)

In [9]:
# The adversary sets a decision threshold by his choise
T = y_score.mean()+ 2.0*y_score.std()
print(T)

0.6908309803634083


In [10]:
# predict that scores exceed the threshold are sensitive training data
labeled_syn['y_pred'] = y_score > T

In [11]:
# The exposed data of TableGAN-MCA are "Positives", the true postive rate are len(TP)/len(Positives)
Positives = labeled_syn[labeled_syn['y_pred']==1]
TP = Positives[Positives['label']==1]
print(len(TP),len(Positives),len(TP)/len(Positives))

2008 2486 0.8077232502011263
